# Set 1

## Task 1: *convert hex to base64*

In [1]:
input_hex = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

In [2]:
import base64
input_bytes = base64.b16decode(input_hex, casefold=True)
print(input_bytes.decode())

I'm killing your brain like a poisonous mushroom


In [3]:
output_b64 = base64.b64encode(input_bytes).decode()
correct_output = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
if correct_output == output_b64:
    print("Task 1 passed successfully!")

Task 1 passed successfully!


Having rewritten much of the code so far into reuseable functions, this task can be refactored:

In [4]:
from functions import *
to_b64(to_bytes(input_hex)) == correct_output

True

## Task 2: *Fixed XOR*

In [5]:
a = "1c0111001f010100061a024b53535009181c"
b = "686974207468652062756c6c277320657965"

Time to get familiar with a great quirk of Python: bit-wise operations take place on *integers*, not on *bytes* objects! So we have to take the input bytes, read them as integers and then get back to hex to check the result.

In [6]:
int_a = int(a, 16)
int_b = int(b, 16)
result = hex(int_a ^ int_b)[2:]

Here, the slicing on the string is to remove the characters that Python naturally adds to the front to let us know this is a hex-encoded string.

In [7]:
correct_output = "746865206b696420646f6e277420706c6179"
if correct_output == result:
    print("Task 2 passed successfully!")


Task 2 passed successfully!


Similarly: (N.B. make sure *functions* is already imported)

In [8]:
to_hex(key_XOR(to_bytes(a), to_bytes(b))) == correct_output

True

## Task 3: *Single-byte XOR cipher*

The first step is to work out what a good English-language plaintext should look like. We can do this by sampling from something like "The Lord of the Rings" to get a Dictionary of letter frequencies:

In [9]:
english_chars = "abcdefghijklmnopqrstuvwxyz "
frequency_dict = {c:0 for c in english_chars}
with open("lotr.txt", "r") as lotr_file:
    total = 0
    for line in lotr_file:
        for c in line:
            if c.lower() in english_chars:
                frequency_dict[c.lower()] += 1
                total += 1

frequency_dict = {c:frequency_dict[c]/total for c in english_chars}

Now, we can loop over the characters in *english_chars* and do the following:
1. generate the XOR of that character with the given ciphertext;
2. compare, using *SciPy*'s built-in *chisquare* test, the letter frequencies recorded for the 'decoded' text with the letter frequencies from "The Lord of the Rings";

and consequently pick the one with the lowest Chi-Square score to be the correct plaintext!

First, we need to get *SciPy*'s *chisquare* function, and can set out some variables for later use:

In [10]:
from scipy.stats import chisquare

inf_ch_sq = 1000
best_decipher = None
key = None

Now we want to decipher the ciphertext, given below as a hex-encoded string:

In [11]:
ciphertext = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
int_ciphertext = int(ciphertext, 16)
buffer_length = len(ciphertext) // 2

In [12]:
keys = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
for c in keys:
    total = 0
    local_dict = {a:0 for a in english_chars}

    buffer = base64.b16encode(c.encode()) * buffer_length
    int_buffer = int(buffer.decode(), 16)

    hex_XOR = hex(int_ciphertext ^ int_buffer)[2:]
    XOR_text = base64.b16decode(hex_XOR, casefold=True).decode()
    
    for d in XOR_text:
        if d.lower() in english_chars:
            local_dict[d.lower()] += 1
            total += 1
    local_dict = {c:local_dict[c]/total for c in english_chars}
    ch_sq = chisquare(list(local_dict.values()), list(frequency_dict.values()))[0]
    if ch_sq < inf_ch_sq:
        inf_ch_sq = ch_sq
        best_decipher = XOR_text
        key = c

print(f"The key was '{key}', with a Chi-Square value of {inf_ch_sq:.2f}")
print("The corresponding plaintext was:")
print(best_decipher)    

The key was 'X', with a Chi-Square value of 1.45
The corresponding plaintext was:
Cooking MC's like a pound of bacon


That concludes task 3! (N.B. 'ETAOIN SHRDLU' is a phrase that occasionally popped up in old newspaper misprints, and is a reference to things that look like total gibberish.)

This gets all nicely wrapped up into "byte_XOR_break", where the key is now given as a hexadecimal number corresponding to the ascii encoding for 'X'.

In [13]:
byte_XOR_break(ciphertext)

('0x58', "Cooking MC's like a pound of bacon", 1.1848580914226594)

Can I explain the discrepancy in the Chi-Square values? Is this just a reflection of a bad counting algorithm implemented by me?

## Task 4: *Detect single-character XOR*

We have a file of hex-encoded strings, one of which is encrypted with a single-byte XOR mask. We can use the same code as for Task 3 to decrypt the strings, and keep any that have a Chi Square coefficient of less than, say, 10, and at least 25 viable letters.

We also need to give some thought to possible keys: whereas above, letters of the alphabet sufficed, here we need to be broader, and consider all possible single-byte keys, i.e. the integers 0 to 255.

In [14]:
keys = ["0" * (2-len(hex(a)[2:])) + hex(a)[2:] for a in range(256)]

In [15]:
viable_strings = {}
with open("s1t4_data.txt", "r") as list_of_strings:
    for s in list_of_strings:
        int_s = int(s[:-3], 16)
        buffer_length = len(s[:-3]) // 2
        for c in keys:
            total = 0
            local_dict = {a:0 for a in english_chars}

            buffer = c * buffer_length
            int_buffer = int(buffer, 16)

            hex_XOR = hex(int_s ^ int_buffer)[2:]

            if len(hex_XOR) % 2 == 1:
                hex_XOR = "0" + hex_XOR
            try:
                XOR_text = base64.b16decode(hex_XOR, casefold=True).decode()
            except:
                continue

            for d in XOR_text:
                if d.lower() in english_chars:
                    local_dict[d.lower()] += 1
                    total += 1
                
            try:
                local_dict = {c:local_dict[c]/total for c in english_chars}
            except:
                continue
            ch_sq = chisquare(list(local_dict.values()), list(frequency_dict.values()))[0]
            if ch_sq < 10 and total > 25:
                viable_strings[s[:-3]] = (c, XOR_text, ch_sq)

viable_strings

{'7b5a4215415d544115415d5015455447414c155c46155f4058455c5b52': ('35',
  'Now that the party is jumping',
  2.4370679438820475)}

That concludes task 4!

## Task 5: *Implement repeating-key XOR*

In repeating key XOR, successive bytes of the plaintext are encrypted with successive bytes of the key. For example, with the example plaintext and key:

In [16]:
plaintext = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
key = "ICE"

As we are using a multi-line plaintext, in some cases we might want to preserve line endings. A "\n" return corresponds to "\x0A", so we can recognise that in the hex-encoded plaintext and just add a return if we find that byte. See the comment in the loop for how this could be used; here it doesn't apply, Cryptopals' linebreaks in the hex-encoded string are purely aesthetic, but the ones in the plaintext need encoding.

In [17]:
base64.b16encode("\n".encode()).decode()

'0A'

We will encrypt the "B" with the "I", the "u" with the "C" and so on. So, we need both objects as hex-encoded bytes, ready to XOR against each other as appropriate.

In [18]:
p_as_hex = base64.b16encode(plaintext.encode()).decode()
key_as_hex = base64.b16encode(key.encode()).decode()

We can start with an empty variable for the ciphertext, but more generally this could be set to an empty list the same length as the plaintext. Then, we need to iterate throught the plaintext, and encode successive bytes with the corresponding bytes of the key.

In [19]:
ciphertext = ""
j = 0
#  print("P  --- K  ---  C")
for i in range(len(p_as_hex) // 2):
    ptext_byte = p_as_hex[2*i:2*i+2]
    # if ptext_byte == "0A":
    #     ciphertext += "\n"
    #     continue
    key_byte = key_as_hex[j % (len(key_as_hex)):j % len(key_as_hex) + 2]
    j += 2

    ptext_int = int(ptext_byte, 16)
    key_int = int(key_byte, 16)
    cipher_byte = hex(ptext_int ^key_int)[2:]
    if len(cipher_byte) % 2 == 1:
        cipher_byte = "0" + cipher_byte
    
    ciphertext += cipher_byte
    # print(f"{ptext_byte} --- {key_byte} --- {cipher_byte}")


In [20]:
correct_ciphertext = """0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"""
if ciphertext == correct_ciphertext:
    print("Task 5 passed correctly!")

Task 5 passed correctly!


Again, this kind of encryption can be *very* neatly wrapped up into a function:

In [21]:
vig_encrypt(plaintext, key) == correct_ciphertext

True

## Task 6: *Break repeating-key XOR*

Apparently this task is going to be a bit harder. Luckily, it's laid out in steps for us, and the first thing to do is be able to correctly compute the Hamming distance between hex-encoded strings. The *hexhamming* library on PyPi can do that for us.

In [22]:
from hexhamming import hamming_distance_string

input1 = "this is a test"
input2 = "wokka wokka!!!"

hex1 = base64.b16encode(input1.encode()).decode()
hex2 = base64.b16encode(input2.encode()).decode()

hamming_distance = hamming_distance_string(hex1, hex2)

In [23]:
correct_hamming_distance = 37
if hamming_distance == correct_hamming_distance:
    print("That worked as expected!")

That worked as expected!


If we take the Hamming distance between blocks of Englis text, we get a distance-per-byte of about 2.7, corresponding to properties of the utf-8 encoding. We can verify this by taking the normalised Hamming distance over blocks of "The Lord of the Rings":

In [27]:
from random import sample

with open('lotr.txt', 'r') as lotr:
        lines_to_read = 1000
        text = ""
        for line in lotr:
            lines_to_read -= 1
            if lines_to_read < 0:
                break
            else:
                for char in line:
                    text += char

for i in sample(range(50), 5):
    print(i, norm_hamming_dist(to_hex(to_bytes(text, 'plaintext')), i))

32 2.6465803452855248
36 2.6427346170872066
49 2.6364994811483915
27 2.6360215276913075
48 2.6447535991140643


By contrast, over blocks of random text (read: encrypted) we get a higher normalised Hamming distance:

In [33]:
for i in sample(range(2, 50), 5):
    print(i, norm_hamming_dist(vig_encrypt(text, 'THIS IS A KEY'), i))

49 3.2101279833967484
42 3.1957887164162586
24 3.32647189021691
16 3.193336468245187
14 3.4576130549025033


However, if we take the normalised Hamming distance over blocks that are the same length as the key, then the XOR cancels out, and we retrieve a plaintext Hamming distance result:

In [35]:
print(13, norm_hamming_dist(vig_encrypt(text, 'THIS IS A KEY'), 13))

13 2.6345393190053383


In [36]:
print(13, norm_hamming_dist(to_hex(to_bytes(text, 'plaintext')), 13))

13 2.6345393190053383


Now we want to try and guess the length of the key. Over blocks of the ciphertext the length of some key, the correct key length should minimise the Hamming distance between the blocks, normalised by the length of the key.

In [39]:
ciphertext = ""
with open("s1t6_data.txt", 'r') as file:
    for s in file:
        for c in s:
            if c != "\n":
                ciphertext += c

ciphertext = to_hex(to_bytes(ciphertext, 'b64'))

In [45]:
hamming_norms = {i: norm_hamming_dist(ciphertext, i) for i in range(2,40)}
sorted_norms = {i: hamming_norms[i] for i in sorted(hamming_norms.keys(), key=lambda x: hamming_norms[x])}
sorted_norms

{29: 2.7593244194229416,
 38: 3.152560455192034,
 13: 3.164335664335664,
 9: 3.1652690426275334,
 16: 3.165379213483146,
 15: 3.1743859649122808,
 22: 3.175123326286117,
 20: 3.182394366197183,
 25: 3.192280701754386,
 32: 3.196377840909091,
 36: 3.1994301994301995,
 26: 3.205010585744531,
 33: 3.2064834390415786,
 7: 3.2074746769123297,
 18: 3.2127285513361463,
 4: 3.21483286908078,
 3: 3.215256008359457,
 11: 3.2157342657342656,
 14: 3.2244397759103642,
 39: 3.2336182336182335,
 34: 3.233876683203402,
 10: 3.234965034965035,
 8: 3.2353351955307263,
 17: 3.241596638655462,
 31: 3.2431761786600495,
 35: 3.2447971781305114,
 21: 3.2465608465608464,
 23: 3.247194950911641,
 2: 3.2505219206680587,
 12: 3.251750700280112,
 30: 3.2578014184397164,
 5: 3.2581881533101047,
 37: 3.261379800853485,
 27: 3.2659611992945328,
 19: 3.2754385964912283,
 24: 3.2824858757062145,
 28: 3.289250353606789,
 6: 3.2900976290097628}

And so we see that the key length is (convincingly) 29!

In [70]:
KEYLENGTH = 29
key = [0] * KEYLENGTH
cipher_bytes = to_bytes(ciphertext)
for i in range(KEYLENGTH):
    relevant_bytes = cipher_bytes[i::29]
    relevant_string = to_hex(relevant_bytes)

    key[i] = byte_XOR_break(relevant_string)[0]


In [72]:
to_plaintext(key)

'Terminator X: Bring the noise'

In [75]:
print(vig_decrypt(ciphertext, to_plaintext(key), 'hex'))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


That concludes Task 6!

## Task 7: *AES in ECB mode*

In [89]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [101]:
ciphertext = ""
with open('s1t7_data.txt', 'r') as file:
    for line in file:
        for char in line:
            if char == "\n":
                continue
            ciphertext += char
cipher_bytes = bytes.fromhex(to_hex(to_bytes(ciphertext, 'b64')))

In [103]:
cipher = Cipher(algorithms.AES(b"YELLOW SUBMARINE"), modes.ECB())
decryptor = cipher.decryptor()
decrypted = decryptor.update(cipher_bytes)
print(decrypted.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


...That concludes Task 7.

## Task 8: *Detect AES in ECB mode*

We don't really have a hint for how to go about this, except that '16 byte plaintext block will always produce the same 16 byte ciphertext'. So, if we break up the ciphertext and find the normed Hamming distance for blocks of length 16, we might find one which is shorter:

In [112]:
with open('s1t8_data.txt', 'r') as file:
    cipher_strings = [string.replace('\n', '') for string in file]


In [114]:
hamming_norms = {s: norm_hamming_dist(s, 16) for s in cipher_strings}
hamming_norms = {s: hamming_norms[s] for s in sorted(hamming_norms.keys(), key=lambda s: hamming_norms[s])}

This produces nothing of note. Instead, we can crawl the strings and see if 16-byte blocks are ever repeated:

In [123]:
from xmlrpc.client import boolean

def block_repeats(s: str) -> boolean:
    s_bytes = to_bytes(s)
    blocks = [to_hex(s_bytes[i*16:(i+1)*16]) for i in range(len(s_bytes) // 16)]
    if len(set(blocks)) == len(blocks):
        return False
    return True

In [125]:
for string in cipher_strings:
    if block_repeats(string):
        print(string)

d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a


The chances of a hex-encoded string having a 16-byte repeat, randomly, is absolutely tiny, so this should be our match! Taking this to be true, we can move on to set 2, where we will in fact learn how to crack ECB encrypted messages, at which point we can come back here and hopefully check our work.

## Appendix A: *Bytes in Python*

To the uninitiated, the *bytes* type in Python can be a little confusing. Turning a string into a *bytes* type is easy:

In [131]:
bytes_example = "Hello World!".encode()
print(bytes_example, type(bytes_example))

b'Hello World!' <class 'bytes'>


Naively, the string just picks up this preceding '**b**', which can be used to initialise *bytes*, too. This object now behaves loosely like a string, in that addition and multiplication simply append other *bytes* to the object:

In [132]:
bytes_example += b" What a wonderful life!"
print(bytes_example)
print(b"Ha" * 5)

b'Hello World! What a wonderful life!'
b'HaHaHaHaHa'


Difficult arises in trying to perform logical operations on *bytes* objects, which aren't defined:

In [133]:
b"Hello" ^ b"World"

TypeError: unsupported operand type(s) for ^: 'bytes' and 'bytes'

*Yes*, this is frustrating. However, we can quickly see that under the hood, *bytes* are not exactly what they seem:

In [134]:
list(b"Hello")

[72, 101, 108, 108, 111]

In fact, *bytes* are just integer lists, where each spot in the list is occupied by a number between 0 and 255 (no big prizes for guessing why). Then, the *bytes* class's real talent is just taking an integer list, and pretty-printing whenever it can the *utf-8* encodings corresponding to the given element.

As such, we now have a very easy way to generate the logical XOR of two *bytes* buffers:

In [135]:
bytes([a ^ b for a, b in zip(b"Hello", b"World")])

b'\x1f\n\x1e\x00\x0b'

Note there are faster ways to implement this, but this is probably the simplest and most Pythonic. As we can now get back on with the cryptography.

**TL; DR**:
- *Bytes* objects are integer lists which Python encodes in the __repr__ dunder so that they display either as ascii characters, or hexadecimal numbers.
- As such, they behave as lists with slicing and addition/multiplication.
- Generating the logical XOR (or other logical operations) of the *bytes* is done using list comprehension: operrating on the integer elements, and then converting back into *bytes*.